In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型花园 - Pic2Word

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_pic2word.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_pic2word.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_pic2word.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

本笔记本演示了如何在Vertex AI Model Garden中使用[Pic2Word](https://github.com/google-research/composed_image_retrieval)模型。

### 目标

在本笔记本中，您将使用预构建的docker映像在Vertex AI上进行实验。

- 在Google Cloud Vertex AI中部署预训练的Pic2Word模型

本教程使用以下Google Cloud ML服务和资源：

- Vertex AI模型注册表
- Vertex AI在线预测

### 数据集

我们使用COCO验证集（5,000张图像）进行评估。

### 成本

本教程使用Google Cloud的收费组件：

- Vertex AI
- 云存储

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing?_ga=2.46650789.-341051769.1686949237)和[云存储定价](https://cloud.google.com/storage/pricing?_ga=2.46650789.-341051769.1686949237)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/?_ga=2.247379078.-341051769.1686949237)根据您的预计使用量生成成本估算。

在你开始之前

仅限Colab
如果您正在使用Workbench或Colab Enterprise，请运行以下命令，并跳过本部分。

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

### 设置您的谷歌云项目

**无论您使用哪种笔记本环境，都需要遵循以下步骤。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您将获得一笔300美元的免费信用额，用于支付计算/存储成本。

2. [确保为您的项目启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. [创建一个用于存储实验输出的Cloud Storage存储桶](https://cloud.google.com/storage/docs/creating-buckets)。

5. [创建一个服务账号](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console)，并为其分配**Vertex AI用户**和**存储对象管理员**角色，用于部署模型到Vertex AI端点。

###导入必要的包

In [ ]:
import os
import sys

from google.cloud import aiplatform

请为实验环境设置以下变量。指定的云存储桶（`BUCKET_URI`）应位于指定的地区（`REGION`）中。请注意，多区域存储桶（例如“us”）不被视为与多区域范围中覆盖的单个地区（例如“us-central1”）匹配。

In [ ]:
# Cloud project ID.
PROJECT_ID = ""  # @param {type:"string"}

# Region for launching jobs.
# TPU deployment is only supported in us-west1.
REGION = ""  # @param {type:"string"}

# Cloud Storage bucket for storing experiment outputs.
# Start with gs:// prefix, e.g. gs://foo_bucket.
BUCKET_URI = "gs://"  # @param {type:"string"}
assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(BUCKET_URI, "gemma")

# Initialize Vertex AI API.
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# The service account looks like:
# '@.iam.gserviceaccount.com'
# Please visit https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create a service account with `Vertex AI User` and `Storage Object Admin` roles.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

# Provision GCS bucket permissions to the SERVICE_ACCOUNT.
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID
! gcloud services enable language.googleapis.com

### 定义常见变量

In [ ]:
# Prediction constants.
PREDICTION_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pic2word-serve:latest"
PREDICTION_ACCELERATOR_TYPE = "NVIDIA_TESLA_T4"
PREDICTION_MACHINE_TYPE = "n1-standard-8"

## 部署模型进行在线预测

这个部分将模型上传到Vertex Model Registry并部署在一个端点资源上。这将大约需要15分钟来完成。

### 上传并部署模型至Vertex AI

In [ ]:
# Upload model.
serving_env = {"MODEL_ID": "googleai-pic2word", "DEPLOY_SOURCE": "notebook"}

model = aiplatform.Model.upload(
    display_name="pic2word-model",
    serving_container_image_uri=PREDICTION_DOCKER_URI,
    serving_container_ports=[7080],
    serving_container_predict_route="/predictions/pic2word",
    serving_container_health_route="/ping",
    serving_container_environment_variables=serving_env,
)

# Create an endpoint.
endpoint = aiplatform.Endpoint.create(display_name="pic2word-endpoint")

# Deploy model to endpoint.
model.deploy(
    endpoint=endpoint,
    machine_type=PREDICTION_MACHINE_TYPE,
    accelerator_type=PREDICTION_ACCELERATOR_TYPE,
    accelerator_count=1,
    traffic_percentage=100,
    service_account=SERVICE_ACCOUNT,
)

您可以在[模型注册表](https://console.cloud.google.com/vertex-ai/models) 中管理您上传的模型，在[终端点](https://console.cloud.google.com/vertex-ai/endpoints) 中管理您的终端点。

向端点发送预测请求

将查询文本设置好并上传您想要查询的图片到指定的INPUT_BUCKET中。在查询文本中使用星号来指定哪个标记被图片标记替换。检索到的图片将上传到指定的输出桶中。

In [ ]:
# Text prompt.
PROMPT = "a bunch of *"

# File name of your query image. Example: "cat.png".
IMAGE_FILE_NAME = "cat.png"

# gs:// path to the directory where the query image is stored.
INPUT_BUCKET = os.path.join(BUCKET_URI, "input")

# gs:// path to the directory where the result images will be uploaded to.
OUTPUT_BUCKET = os.path.join(BUCKET_URI, "output")

instances = [
    {
        "prompts": PROMPT,
        "image_path": INPUT_BUCKET,
        "image_file_name": IMAGE_FILE_NAME,
        "output_storage_dir": OUTPUT_BUCKET,
    },
]

response = endpoint.predict(instances=instances)
print(response)

## 清理工作

要清理此项目中使用的所有 Google Cloud 资源，您可以删除用于本教程的 Google Cloud 项目。

否则，您可以删除您在本教程中创建的各个资源。

In [ ]:
# Delete endpoint resource.
endpoint.delete(force=True)

# Delete model resource.
model.delete()

# Delete Cloud Storage objects that were created.
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI